In [40]:
import os
import OpenDartReader
import FinanceDataReader as fdr

In [41]:
stock_list = fdr.StockListing("KRX").dropna()

### openDART 한번에 모으기

In [42]:
stock_list = stock_list.loc[stock_list['Market'].isin([ "KOSDAQ"])] # Market 기준 필터링
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),"Name"].tolist() # 선물과 같이 KOSPI에는 있으나 배당이 없을수밖에 없는 종목은 제외
import OpenDartReader
my_api = "6f1dcbd821a73d922006b9b9efeac4eebf97df13"
dart = OpenDartReader(my_api)

In [43]:
stock_name_list

['3S',
 'APS홀딩스',
 'AP시스템',
 'AP위성',
 'BNGT',
 'CBI',
 'CJ ENM',
 'CJ 바이오사이언스',
 'CJ프레시웨이',
 'CMG제약',
 'CNH',
 'CNT85',
 'CS',
 'CSA 코스믹',
 'DMS',
 'DSC인베스트먼트',
 'EDGC',
 'EG',
 'ES큐브',
 'EV수성',
 'FSN',
 'GH신소재',
 'GST',
 'HB솔루션',
 'HB테크놀러지',
 'HK이노엔',
 'HLB',
 'HLB생명과학',
 'HLB제약',
 'HRS',
 'IBKS제12호스팩',
 'IBKS제16호스팩',
 'ISC',
 'ITX-AI',
 'JTC',
 'JW신약',
 'JYP Ent.',
 'KBG',
 'KBI메탈',
 'KB오토시스',
 'KCC건설',
 'KCI',
 'KD',
 'KG ETS',
 'KG모빌리언스',
 'KG이니시스',
 'KH 전자',
 'KH바텍',
 'KMH',
 'KMH하이텍',
 'KNN',
 'KPX생명과학',
 'KT서브마린',
 'LX세미콘',
 'NEW',
 'NE능률',
 'NHN벅스',
 'NHN한국사이버결제',
 'NICE평가정보',
 'PN풍년',
 'RFHIC',
 'RF머트리얼즈',
 'S&K폴리텍',
 'SBI인베스트먼트',
 'SBI핀테크솔루션즈',
 'SBS콘텐츠허브',
 'SBW생명과학',
 'SCI평가정보',
 'SDN',
 'SFA반도체',
 'SG',
 'SG&G',
 'SGA',
 'SGA솔루션즈',
 'SGC이테크건설',
 'SM C&C',
 'SM Life Design',
 'SNK',
 'SV인베스트먼트',
 'THE E&M',
 'THE MIDONG',
 'THQ',
 'TJ미디어',
 'TKG애강',
 'TPC',
 'TS인베스트먼트',
 'UCI',
 'WI',
 'YBM넷',
 'YTN',
 'YW',
 'iMBC',
 '가비아',
 '가온미디어',
 '감성코퍼레이션',
 '강스템바이오텍',
 '강원에너지',
 '갤럭시

In [52]:
import numpy as np
import pandas as pd
def find_financial_indicators(stock_name, year, indicators):
    report = dart.finstate(stock_name, year) # 데이터 가져오기
    if report is None: # 리포트가 없다면 (참고: 리포트가 없으면 None을 반환함)
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[stock_name, year] + [np.nan] * len(indicators)]
        data = [[stock_name, year-1] + [np.nan] * len(indicators)]
        data = [[stock_name, year-2] + [np.nan] * len(indicators)]
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else:
        report = report[report['account_nm'].isin(indicators)] # 관련 지표로 필터링
        if sum(report['fs_nm'] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표를 사용
            report = report.loc[report['fs_nm'] == "연결재무제표"]
            
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report['fs_nm'] == "재무제표"]
        
        data = []
        for y, c in zip([year, year-1], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report['account_nm'] == indic) > 0: # 해당 지표가 있다면 추가 (지표가 없는 경우도 있음)
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else:
                    value = np.nan
                
                record.append(value)
            
            data.append(record)
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)

In [22]:
result = dart.finstate("LG전자", 2020, "11011") # 2020년 LG전자 사업보고서 내 재무 정보 가져오기
result = result.loc[result['fs_nm'] == "연결재무제표"] # 연결재무제표 필터링
# account_nm이 당기순이익, 영업이익, 매출액 중 하나인 경우에만 가져오기
result = result.loc[result['account_nm'].isin(['당기순이익', '영업이익', '매출액'])] 

# 열 필터링
result = result.loc[:, ["account_nm", "fs_nm", "sj_nm", "thstrm_dt", "thstrm_amount"]]


In [53]:
import time 
indicators = ['유동자산', '비유동자산','자산총계','유동부채','비유동부채', '부채총계', '자본금','이익잉여금','자본총계', '매출액', '영업이익', '당기순이익']
data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정

for idx, stock_name in enumerate(stock_name_list):
    print(idx+1, "/", len(stock_name_list)) # 현재까지 진행된 상황 출력
    for year in [2018, 2019, 2020]:
        try: # 재무제표 데이터가 잘 불러와지지 않는 경우가 있어, try - except으로 넘김
            result = find_financial_indicators(stock_name, year, indicators) # 재무지표 데이터 
        except:
            pass
        data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
        time.sleep(0.5)

1 / 1431
2 / 1431
3 / 1431
4 / 1431
5 / 1431
6 / 1431
7 / 1431
8 / 1431
9 / 1431
10 / 1431
11 / 1431
12 / 1431
13 / 1431
14 / 1431
15 / 1431
16 / 1431
17 / 1431
18 / 1431
19 / 1431
20 / 1431
21 / 1431


KeyboardInterrupt: 

In [47]:
# 중복된 행 제거: 
data.drop_duplicates(inplace = True)

In [48]:
# 숫자로 모두 변환
def str_to_float(value):
    if type(value) == float: # nan의 경우에는 문자가 아니라, 이미 float 취급됨
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

for indc in indicators:
    data[indc] = data[indc].apply(str_to_float)

###
   * 부채비율 = 자본 총계 / 자산 총계
   * 영업이익 증가율 = (당기 영업이익 - 전기 영업이익) / 전기 영업이익 * 100%

   * 영업이익/매출액/당기순이익 상태

   * ROA = (당기순이익 / 자산총계) * 100%

   * ROE = (당기순이익 / 평균 자기 자본)

In [49]:

data['부채비율'] = data['자본총계'] / data['자산총계'] * 100
data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['영업이익증가율'] = data['영업이익'].diff() / data['영업이익']


data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['매출액증가율'] = data['매출액'].diff() / data['매출액']


data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['당기순이익증가율'] = data['당기순이익'].diff() / data['당기순이익']


data['ROA'] = data['당기순이익'] / data['자산총계']

average_equity = data['자본총계'].rolling(2).mean() # 평균 자기 자본

data['ROE'] = data['당기순이익'] / average_equity

In [50]:
data.to_csv("./코스닥주요재무지표6.csv", index = False,encoding='CP949')